In [1]:
# Importamos las librerías que necesitamos
# 
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [8]:
df_c = pd.read_csv('clientes.csv')
df_p = pd.read_csv('productos.csv')
df_v = pd.read_csv('ventas.csv')

In [9]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_producto  600 non-null    object
 1   precio       600 non-null    object
 2   categoria    600 non-null    object
 3   descripcion  600 non-null    object
dtypes: object(4)
memory usage: 18.9+ KB


In [12]:
df_v.isnull().sum()

id_venta       0
id_producto    0
cantidad       0
fecha          0
id_cliente     0
dtype: int64

In [10]:
df_p.duplicated().sum()

0

In [11]:
df_p

,id_producto,precio,categoria,descripcion
0,P0001,58.91 €,Clothing,6GJFfPug1voQYH3zfXItpbjwRA7VVT64DGtG8VnRXaJJZ0SYJa
1,P0002,15.67 €,Electronics,Ai3c6EdaRU5C0WuWl718dSYHakqGLiMJmpbVgn7iq6syGN3UJP
2,P0003,50.58 €,Sports & Outdoors,yMidjq8PGxKEf6YFWfUKJN1sDhyIs2xareF0uaA4ngzUgmrDKc
3,P0004,72.88 €,Sports & Outdoors,E8LBdxZItLULWZUyKPwzkjZJHD2GOSuqR7Od4DX5624ex1t5lN
4,P0005,70.23 €,Books,zQLYUssRkFNu2GcCd8xBPrc5xiJH5DGQ66A0X4TFU0cHeazM6o
...,...,...,...,...
595,P0596,14.50 €,Home & Kitchen,ooFtlWXZfdN8Lo75Nvdbc0yJpOKbyuSepeKWfjEsagEpNLqU3W
596,P0597,34.23 €,Electronics,MVubc35s8NZOjMuGchcaDv53uAAA3UYu6iarCEiSmQEFwy2vd5
597,P0598,31.16 €,Books,ajscjQ9ZQBPQ5eAzbHJXNv1xG8lSYXFi1Zyi66NeAUdU22gZ1s
598,P0599,72.33 €,Clothing,msikU2NDRKRgfapWE19Kp6QJRdywxHgl81xVR7duwHw1lh8qOI


Termino de analizar

In [13]:
df_p.describe().T

,count,unique,top,freq
id_producto,600,600,P0001,1
precio,600,585,58.91 €,3
categoria,600,5,Clothing,136
descripcion,600,600,6GJFfPug1voQYH3zfXItpbjwRA7VVT64DGtG8VnRXaJJZ0SYJa,1


In [14]:
df_p.describe(include='object').T

,count,unique,top,freq
id_producto,600,600,P0001,1
precio,600,585,58.91 €,3
categoria,600,5,Clothing,136
descripcion,600,600,6GJFfPug1voQYH3zfXItpbjwRA7VVT64DGtG8VnRXaJJZ0SYJa,1


Elimino el símbolo del €

In [15]:
df_p['precio'] = df_p['precio'].str.replace('€', '').astype(float)

Compruebo el cambio

In [16]:
df_p

,id_producto,precio,categoria,descripcion
0,P0001,58.91,Clothing,6GJFfPug1voQYH3zfXItpbjwRA7VVT64DGtG8VnRXaJJZ0SYJa
1,P0002,15.67,Electronics,Ai3c6EdaRU5C0WuWl718dSYHakqGLiMJmpbVgn7iq6syGN3UJP
2,P0003,50.58,Sports & Outdoors,yMidjq8PGxKEf6YFWfUKJN1sDhyIs2xareF0uaA4ngzUgmrDKc
3,P0004,72.88,Sports & Outdoors,E8LBdxZItLULWZUyKPwzkjZJHD2GOSuqR7Od4DX5624ex1t5lN
4,P0005,70.23,Books,zQLYUssRkFNu2GcCd8xBPrc5xiJH5DGQ66A0X4TFU0cHeazM6o
...,...,...,...,...
595,P0596,14.50,Home & Kitchen,ooFtlWXZfdN8Lo75Nvdbc0yJpOKbyuSepeKWfjEsagEpNLqU3W
596,P0597,34.23,Electronics,MVubc35s8NZOjMuGchcaDv53uAAA3UYu6iarCEiSmQEFwy2vd5
597,P0598,31.16,Books,ajscjQ9ZQBPQ5eAzbHJXNv1xG8lSYXFi1Zyi66NeAUdU22gZ1s
598,P0599,72.33,Clothing,msikU2NDRKRgfapWE19Kp6QJRdywxHgl81xVR7duwHw1lh8qOI


Elimino la columna 'descripcion'

In [18]:
df_p.drop('descripcion', axis=1, inplace=True)

Compruebo los cambios

In [19]:
df_p

,id_producto,precio,categoria
0,P0001,58.91,Clothing
1,P0002,15.67,Electronics
2,P0003,50.58,Sports & Outdoors
3,P0004,72.88,Sports & Outdoors
4,P0005,70.23,Books
...,...,...,...
595,P0596,14.50,Home & Kitchen
596,P0597,34.23,Electronics
597,P0598,31.16,Books
598,P0599,72.33,Clothing


Exporto el dataframe limpio de productos a .csv 

In [22]:
df_p.to_csv('productos_limpio.csv')